In [ ]:
# You need to install the OpenSkyAPI library
# Get it here: https://github.com/openskynetwork/opensky-api

In [ ]:
# Install it
#!pip install -e \opensky-api-master\python

In [ ]:
from opensky_api import OpenSkyApi
import geocoder
import pandas as pd
import json
from pandas.io.json import json_normalize
import requests

In [ ]:
# Get your own Lat/Long position
g = geocoder.ip('me')
#print(g.latlng)

In [ ]:
# Add a bounding box of 1 degree (I know that this is the lazy solution)

lat1 = g.latlng[0]-0.5
lat2 = g.latlng[0]+0.5
# print (lat1, lat2)

In [ ]:
lng1 = g.latlng[1]-0.5
lng2 = g.latlng[1]+0.5
# print (lng1, lng2)

In [ ]:
# API request
api = OpenSkyApi()
states = api.get_states(bbox=(lat1,lat2,lng1,lng2))


In [ ]:
# Delete your pre-existing dataframe
try:
    del df
except: Exception
        
    

# Create your new Dataframe
df = pd.DataFrame(columns=['longitude', 'latitude', 'velocity', 'callsign', 'origin_country', 'on_ground', 'squawk', 'vertical_rate', 'icao24'])

In [ ]:
# Fill your dataframe - again, lazy solution, 'cause the read_json didn't work

for s in states.states:
    df.loc[s] = [ s.longitude, s.latitude, s.velocity, s.callsign, s.origin_country, s.on_ground, s.squawk, s.vertical_rate, s.icao24 ]
df.reset_index(inplace=True)

In [ ]:
df

In [ ]:
url = "https://ae.roplan.es/api/hex-type.php?hex="
headers = {
    'User-Agent': 'Mozilla/5.0 (iPad; U; CPU OS 3_2_1 like Mac OS X; en-us'
}

In [ ]:
# Function to get the aircraft type from ae.roplan.es
# I know, you wouldn't need it here but I used a different website with BS4 etc
def get_type(callsign):
    r = requests.get(url+callsign, headers=headers)
    return (r.text)

In [ ]:
# Test it
# get_type("49d027")

In [ ]:
# Add aircraft type to dataframe
for index,row in df.iterrows():
    df.loc[index,'type'] = get_type(row['icao24'])

In [ ]:
# Filter dataframe by aircraft type
df[df['type'].astype(str).str.contains('Boeing 737')]